# Prerequisites

To use SageMaker AI endpoints in these examples, you will need to first deploy a managed endpoint. In this example you will deploy an endpoint through SageMaker Jumpstart, a feature that helps machine learning practitioners quickly get started with hundreds of production-ready models in SageMaker AI.

## Dependencies (Warnings are safe to ignore)

In [ ]:
%pip uninstall -q -y autogluon-multimodal autogluon-timeseries autogluon-features autogluon-common autogluon-core
%pip install -Uq sagemaker==2.239.0
%pip install -Uq boto3==1.38.33
%pip install -Uq litellm==1.72.2
%pip install -Uq aiohttp==3.12.11

## This cell will restart the kernel. Wait for the pop-up box to appear, then click "OK" before proceeding.

In [ ]:
from IPython import get_ipython
get_ipython().kernel.do_shutdown(True)

## Deploy the model from SageMaker JumpStart on a SageMaker Inference endpoint

> Note: skip the cell below if you have already deployed your model.

In [ ]:
from sagemaker.djl_inference import DJLModel
import sagemaker

model_id = "Qwen/Qwen3-1.7B"
model_name = sagemaker.utils.name_from_base(model_id.split("/")[0].replace(".","p"))
model = DJLModel(
    name=model_name,
    image_uri="763104351884.dkr.ecr.us-east-1.amazonaws.com/djl-inference:0.33.0-lmi15.0.0-cu128-v1.3",
    role=sagemaker.get_execution_role(),
    env={
        "HF_MODEL_ID": model_id, # config: https://qwen.readthedocs.io/en/latest/framework/function_call.html#vllm 
        "OPTION_MAX_MODEL_LEN": f"{1024*32}",
        # vllm serve {model_id} --enable-auto-tool-choice --tool-call-parser hermes
        "OPTION_ROLLING_BATCH": "vllm",
        "OPTION_ENABLE_AUTO_TOOL_CHOICE": "true",
        "OPTION_TOOL_CALL_PARSER": "hermes",
        # --enable-reasoning --reasoning-parser deepseek_r1
        # "OPTION_ENABLE_REASONING": "true",
        # "OPTION_REASONING_PARSER": "qwen3" # currently not available in djl lmi15
    }
)
model.deploy(
    endpoint_name=sagemaker.utils.name_from_base("qwen3-ep"),
    initial_instance_count=1,
    instance_type="ml.g5.xlarge",
)

In [ ]:
SAGEMAKER_ENDPOINT_NAME = predictor.endpoint_name
print(f"Endpoint name: {SAGEMAKER_ENDPOINT_NAME}")

%store SAGEMAKER_ENDPOINT_NAME

<div class="alert alert-block alert-info">
⚠️ <b>Note:</b> deployment will take 5~7 minutes. Take note of the endpoint name and the inference component names, as they will be needed later.
</div>